<a href="https://colab.research.google.com/github/Kanishka-Jingar/Machine_Learning-Lab/blob/main/Experiment/Experiment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install --upgrade pgmpy  # Ensure pgmpy is up-to-date
from pgmpy.models import DiscreteBayesianNetwork # Import DiscreteBayesianNetwork instead of BayesianNetwork or BayesianModel

cancer_model = DiscreteBayesianNetwork([('Pollution', 'Cancer'),
                              ('Smoker', 'Cancer'),
                              ('Cancer', 'Xray'),
                              ('Cancer', 'Dyspnoea')])
print(cancer_model)

DiscreteBayesianNetwork with 5 nodes and 4 edges


In [24]:
print(cancer_model)

DiscreteBayesianNetwork with 5 nodes and 4 edges


In [4]:
cancer_model.nodes()

NodeView(('Pollution', 'Cancer', 'Smoker', 'Xray', 'Dyspnoea'))

In [5]:
cancer_model.edges()

OutEdgeView([('Pollution', 'Cancer'), ('Cancer', 'Xray'), ('Cancer', 'Dyspnoea'), ('Smoker', 'Cancer')])

In [6]:
cancer_model.get_cpds()
#conditional probabilities

[]

In [25]:
#creation of condition probability table

from pgmpy.factors.discrete import TabularCPD

In [26]:
cpd_poll = TabularCPD(variable='Pollution', variable_card=2,
                      values=[[0.9],[0.1]])
cpd_smoke = TabularCPD(variable='Smoker', variable_card=2,
                      values=[[0.3],[0.7]])
cpd_cancer = TabularCPD(variable='Cancer', variable_card=2,
                      values=[[0.03, 0.05, 0.001, 0.02],
                      [0.97, 0.95, 0.999, 0.98]],
                      evidence=['Smoker', 'Pollution'],
                      evidence_card=[2, 2])
cpd_xray = TabularCPD(variable='Xray', variable_card=2,
                      values=[[0.9, 0.2],[0.1, 0.8]],
                      evidence=['Cancer'], evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea', variable_card=2,
                      values=[[0.65, 0.3],[0.35, 0.7]],
                      evidence=['Cancer'], evidence_card=[2])

In [27]:
cancer_model.add_cpds(cpd_poll,cpd_smoke,cpd_cancer,cpd_xray,cpd_dysp)

#check if cpds are valid
cancer_model.get_cpds()
cancer_model.check_model()

True

In [28]:
#If you want to stick with DiscreteBayesianNetwork and identify active trails.
#This approach identifies all nodes on the active trail, including the start and end nodes.
active_trail = cancer_model.active_trail_nodes('Pollution', observed=['Cancer'])
# Check if 'Smoker' is in the active trail
is_active = 'Smoker' in active_trail['Pollution']
print(f"Is there an active trail between Pollution and Smoker given Cancer? {is_active}")


Is there an active trail between Pollution and Smoker given Cancer? True


In [29]:
cancer_model.local_independencies('Xray')

(Xray ⟂ Smoker, Pollution, Dyspnoea | Cancer)

In [30]:
cancer_model.local_independencies('Pollution')

(Pollution ⟂ Smoker)

In [31]:
cancer_model.local_independencies('Smoker')

(Smoker ⟂ Pollution)

In [32]:
cancer_model.local_independencies('Dyspnoea')

(Dyspnoea ⟂ Smoker, Pollution, Xray | Cancer)

In [34]:
cancer_model.local_independencies('Cancer')

In [35]:
cancer_model.local_independencies('Dyspnoea')

(Dyspnoea ⟂ Smoker, Pollution, Xray | Cancer)

In [36]:
cancer_model.local_independencies('Pollution')

(Pollution ⟂ Smoker)

In [37]:
cancer_model.get_independencies()


(Dyspnoea ⟂ Pollution | Cancer)
(Smoker ⟂ Dyspnoea | Cancer)
(Smoker ⟂ Xray | Cancer)
(Dyspnoea ⟂ Xray | Cancer)
(Smoker ⟂ Pollution)
(Pollution ⟂ Xray | Cancer)

In [38]:
cancer_model.get_cpds()
print(cancer_model.get_cpds('Pollution'))

+--------------+-----+
| Pollution(0) | 0.9 |
+--------------+-----+
| Pollution(1) | 0.1 |
+--------------+-----+


In [39]:
cancer_model.get_cpds()
print(cancer_model.get_cpds('Cancer'))

+-----------+--------------+--------------+--------------+--------------+
| Smoker    | Smoker(0)    | Smoker(0)    | Smoker(1)    | Smoker(1)    |
+-----------+--------------+--------------+--------------+--------------+
| Pollution | Pollution(0) | Pollution(1) | Pollution(0) | Pollution(1) |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(0) | 0.03         | 0.05         | 0.001        | 0.02         |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(1) | 0.97         | 0.95         | 0.999        | 0.98         |
+-----------+--------------+--------------+--------------+--------------+


In [40]:
cancer_model.get_cpds()
#conditional probabilities

[<TabularCPD representing P(Pollution:2) at 0x79493f52ce90>,
 <TabularCPD representing P(Smoker:2) at 0x794945cd2790>,
 <TabularCPD representing P(Cancer:2 | Smoker:2, Pollution:2) at 0x79493f52f290>,
 <TabularCPD representing P(Xray:2 | Cancer:2) at 0x79493f52e790>,
 <TabularCPD representing P(Dyspnoea:2 | Cancer:2) at 0x79493f52e4d0>]

In [41]:
#Inferencing with Bayesian Network with Variable Elimination
from pgmpy.inference import VariableElimination
cancer_infer = VariableElimination(cancer_model)

q=cancer_infer.query(variables=['Cancer'], evidence={'Smoker': 1})
print(q)

+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0029 |
+-----------+---------------+
| Cancer(1) |        0.9971 |
+-----------+---------------+


In [42]:
r=cancer_infer.query(variables=['Cancer'], evidence={'Smoker': 1, 'Pollution':1})
print(r)

+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0200 |
+-----------+---------------+
| Cancer(1) |        0.9800 |
+-----------+---------------+


In [43]:
s=cancer_infer.query(variables=['Cancer'], evidence={'Pollution':1})
print(r)

+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0200 |
+-----------+---------------+
| Cancer(1) |        0.9800 |
+-----------+---------------+


In [44]:
cancer_model.get_cpds()
print(cancer_model.get_cpds('Smoker'))

+-----------+-----+
| Smoker(0) | 0.3 |
+-----------+-----+
| Smoker(1) | 0.7 |
+-----------+-----+


In [13]:
cancer_model.get_cpds()
print(cancer_model.get_cpds('Xray'))

+---------+-----------+-----------+
| Cancer  | Cancer(0) | Cancer(1) |
+---------+-----------+-----------+
| Xray(0) | 0.9       | 0.2       |
+---------+-----------+-----------+
| Xray(1) | 0.1       | 0.8       |
+---------+-----------+-----------+


In [14]:
cancer_model.get_cpds()
print(cancer_model.get_cpds('Dyspnoea'))

+-------------+-----------+-----------+
| Cancer      | Cancer(0) | Cancer(1) |
+-------------+-----------+-----------+
| Dyspnoea(0) | 0.65      | 0.3       |
+-------------+-----------+-----------+
| Dyspnoea(1) | 0.35      | 0.7       |
+-------------+-----------+-----------+
